In [1]:
# Importanto bibliotecas utilizadas
import pandas as pd
import numpy as np
import re

In [2]:
# Importando o dataset
aptos = pd.read_csv('aptos.csv')
aptos.head()

,Unnamed: 0,titulo,endereco,preco,quartos,suites,banheiros,vagas,privativos,total,infra,coordenadas,link
0,0,Apartamento 2 Quartos Meia Praia 61m²,"Rua Geraldo José Borba, 130, Meia Praia - Nave...","R$ 1.400,00",2,NaN,1.0,1.0,61 M2,68 M2,[],https://www.google.com/maps/embed/v1/place?key...,https://maximoveis.com.br/imovel/9657/apartame...
1,1,Apartamento 3 Quartos Fazenda 79m²,"Rua Lauro Muller, 1034, Fazenda - Itajaí/SC","R$ 880.000,00",3,1.0,2.0,2.0,79 M2,105 M2,"['Bicicletário', 'Brinquedoteca', 'Condomínio ...",https://www.google.com/maps/embed/v1/place?key...,https://maximoveis.com.br/imovel/12845/apartam...
2,2,Apartamento 2 Quartos São João 69m²,"Rua Heitor Liberato, 2150, São João - Itajaí/SC","R$ 490.000,00",2,1.0,2.0,1.0,69 M2,94 M2,"['Condomínio Fechado', 'Elevador', 'Piscina Co...",https://www.google.com/maps/embed/v1/place?key...,https://maximoveis.com.br/imovel/12846/apartam...
3,3,Apartamento 2 Quartos Itaipava 49m²,"Avenida Itaipava, 3868, Itaipava - Itajaí/SC","R$ 255.000,00",2,NaN,1.0,1.0,49 M2,53 M2,"['Condomínio Fechado', 'Estacionamento', 'Guar...",https://www.google.com/maps/embed/v1/place?key...,https://maximoveis.com.br/imovel/8174/apartame...
4,4,Apartamento 3 Quartos Centro 72m²,"Rua Luiz Berlim, 165, Centro - Itajaí/SC","R$ 420.000,00",3,NaN,1.0,1.0,72 M2,85 M2,['Salão de Festas'],https://www.google.com/maps/embed/v1/place?key...,https://maximoveis.com.br/imovel/12821/apartam...


In [3]:
# Verificando o tamanho do dataset:
aptos.shape
print(f'Atualmente o dataset está com {aptos.shape[0]} linhas e {aptos.shape[1]} colunas.')

Atualmente o dataset está com 400 linhas e 13 colunas.


In [4]:
# Verificando os tipos de dados que estão nas colunas:
aptos.dtypes

Unnamed: 0       int64
titulo          object
endereco        object
preco           object
quartos          int64
suites         float64
banheiros      float64
vagas          float64
privativos      object
total           object
infra           object
coordenadas     object
link            object
dtype: object

In [5]:
# Verificando a presença de valores nulos nas colunas:
aptos.isnull().sum()

Unnamed: 0       0
titulo           0
endereco         0
preco            0
quartos          0
suites          67
banheiros        2
vagas            5
privativos       2
total          123
infra            0
coordenadas     10
link             0
dtype: int64

In [6]:
# Função para limpar a variável preço: 
def limpar_preco(preco):
    # Retirar da frente o 'R$'
    preco_sem_r = re.sub(r'^R\$','', preco)
    # Retirar os pontos:
    preco_sem_pontos = re.sub(r'\.','', preco_sem_r)
    # Substituir a vírgula por poonto e retornar no retormato float
    preco_em_float = re.sub(r',','.', preco_sem_pontos)
    return float(preco_em_float)
# Aplicando na variável preço:
aptos.preco = aptos.preco.apply(limpar_preco)


In [7]:
# Função para limpar as variáveis que determinam área e possuem "M2" (metro quadrado) no final.
def extrair_metros_quadrados(texto):
    # Se for nulo, vai continuar sendo nulo
    metragem = texto
    # Tenta retirar o 'M2' do final para os dados que apresentarem e transforma para inteiro.
    try:
        match = re.search(r'(\d+) M2', texto)
        metragem = int(match.group(1)) 
    except: 
        pass
    return metragem
# Aplica a função na variável 'privativos' e na variável 'totais'
aptos.privativos = aptos.privativos.apply(extrair_metros_quadrados)
aptos.total = aptos.total.apply(extrair_metros_quadrados)

In [8]:
# Função para extrair as coordenadas dos links do Google Maps:
def extrair_coordenadas(link):
    # Se não possuir a informação, returna nulo.
    if link is None or pd.isna(link):
        return None, None
    # Padrão da regex
    padrao = r'q=(-?\d+\.\d+), (-?\d+\.\d+)'
    # Busca pelo padrão e salva as duas informações de latitude e longitude
    dados = re.search(padrao, link)
    # Defini as novas variáveis e retorna a dupla:
    if dados:
        latitude = float(dados.group(1))
        longitude = float(dados.group(2))
        return latitude, longitude
    else:
        return None, None
# Cria duas novas colunas (lat e long) para receberem os valores de latitude e longitude.
aptos['lat'], aptos['long'] = zip(*aptos['coordenadas'].map(extrair_coordenadas))



In [9]:
# Função para separar o endereço em 'Rua com número', 'Bairro' e 'Cidade'
def separar_endereco(endereco):
    # Inicializa as variáveis para as partes do endereço
    numero_rua = None
    bairro = None
    cidade = None

    # Use regex para extrair as partes do endereço
    dados = re.match(r'^(.*?), (\d+), (.*?) - (.*?)$', endereco)
    # Armazenar os valores e retorna:
    if dados:
        numero_rua = dados.group(1) + ", " + dados.group(2)
        bairro = dados.group(3)
        cidade = dados.group(4)
    return numero_rua, bairro, cidade
# Cria três novas colunas para receberem as três informações:
aptos['numero_rua'], aptos['bairro'], aptos['cidade'] = zip(*aptos['endereco'].apply(separar_endereco))


In [111]:
# Verifica as cidades que estão presentes os imóveis:
aptos.cidade.value_counts()

Itajaí/SC                292
Navegantes/SC             61
Balneário Camboriú/SC     46
Camboriú/SC                1
Name: cidade, dtype: int64

In [10]:
# Junta todos os itens de infraestrutura dos condomínios em um único conjunto:
aptos['infra'] = aptos['infra'].apply(lambda x: eval(x) if isinstance(x, str) else x)

infra_total = list(set(item for lista in aptos['infra'] for item in lista))

In [11]:
# Verificar os condomínios que possuem 'Piscina Coletiva' na lista de infraestrutura:
def check_piscina(lista):
    # Define como 0 a ausência de piscina
    valor = 0
    # Varre as linhas da coluna buscando por piscina e caso seja verdadeiro atribui 1
    if 'Piscina Coletiva' in lista:
        valor = 1
    return valor
# Cria a coluna booleana da piscina.
aptos['piscina'] = aptos.infra.apply(check_piscina)

In [12]:
# Semelhante ao caso anterior, verifica a presença de 'Gás Central' OU 'Aquecimento Central' nos condomínios:
def check_gas(lista):
    valor = 0
    if 'Gás Central' in lista or 'Aquecimento Central' in lista:
        valor = 1
    return valor
# Cria a nova coluna 'gas_central' com 0 para ausência e 1 para presença:
aptos['gas_central'] = aptos.infra.apply(check_gas)

In [13]:
# Função para verificar se o condomínio possui elevador:
def check_elevador(lista):
    valor = 0
    if 'Elevador' in lista:
        valor = 1
    return valor
# Cria a variável 'elevador' e determina a presença ou não de elevadores no prédio:
aptos['elevador'] = aptos.infra.apply(check_elevador)

In [14]:
# Função para verificar se o condomínio possui 'Salão de Festas':
def check_salao(lista):
    valor = 0
    if 'Salão de Festas' in lista:
        valor = 1
    return valor
# Cria a nova variável 'salao':
aptos['salao'] = aptos.infra.apply(check_salao)

In [15]:
# Função para verificar a presença de 'Academia' ou 'Sala Fitness' no condomínio:
def check_academia(lista):
    valor = 0
    if 'Sala Fitness' in lista or 'Academia' in lista:
        valor = 1
    return valor
# Cria a nova variável 'academia':
aptos['academia'] = aptos.infra.apply(check_academia)

In [18]:
# Retirando os apartamentos que possuem mais de 5 quartos, são pouquíssimos casos (3) e não temos interesse nos
# apartamentos que fogem muito do padrão. Um deles, inclusive, na verdade é um conjunto de kitnets.

aptos = aptos[aptos['quartos']<=5]


C:\Users\pedri\AppData\Local\Temp\ipykernel_24428\3092448350.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  aptos.max()


Unnamed: 0                                                   398
titulo                   Apartamento 5 Quartos Praia Brava 350m²
endereco        Rua Áustria, 144, Nações - Balneário Camboriú/SC
preco                                                 14895000.0
quartos                                                        5
suites                                                       5.0
banheiros                                                    6.0
vagas                                                       21.0
privativos                                                 519.0
total                                                      689.0
infra          [Água, Brinquedoteca, Churrasqueira Condominia...
link           https://maximoveis.com.br/imovel/9947/apartame...
lat                                                    -26.85471
long                                                  -48.629043
numero_rua                                      Rua Áustria, 144
bairro                   

In [20]:
aptos.vagas.value_counts()
# Retirar um imóvel que possui 21 vagas de garagem. Isso está bem fora na normalidade:
aptos = aptos[aptos['vagas'] <= 6]

In [39]:
# Vamos retirar os imóveis muito luxuosos e possuem valores extremamente altos utilizando a remoção de outliers por IQR.
q1_preco = aptos.preco.quantile(.25)
q3_preco = aptos.preco.quantile(.75)
iqr_preco = q3_preco - q1_preco

inf_preco = q1_preco - (1.5*iqr_preco)
sup_preco = q3_preco + (1.5*iqr_preco)

print(f'IQR Preco: {iqr_preco}.')
print(f'Limite Inferior: {inf_preco}.')
print(f'Limite Superior: {sup_preco}.')

outliers_preco = len(aptos[aptos.preco > sup_preco])
print(f'Outliers de preço: {outliers_preco}.')
print(f'% do dataset perdida em outliers {round(outliers_preco/aptos.shape[0],3)*100}%.')

aptos_copy = aptos.copy()
aptos_copy.drop(aptos[aptos.preco > sup_preco].index, axis = 0, inplace = True)

IQR Preco: 1160409.1725.
Limite Inferior: -1212687.99625.
Limite Superior: 3428948.69375.
Outliers de preço: 31.
% do dataset perdida em outliers 7.9%.


In [40]:
# Exclui algumas alunos que não são mais necessárias:
df_limpo = aptos_copy.drop(['Unnamed: 0', 'endereco','infra','coordenadas',], axis = 1)

In [41]:
# Tamanho do dataset atualmente:
df_limpo.shape

(361, 19)

In [42]:
# Filtra apenas os apartamentos na cidade de Itajaí/SC:
df_limpo_itajai = df_limpo.loc[df_limpo['cidade'] == 'Itajaí/SC']

In [43]:
# Verifica a presença de valores nulos em cada coluna:
df_limpo_itajai.isnull().sum()

titulo          0
preco           0
quartos         0
suites         49
banheiros       1
vagas           0
privativos      0
total          70
link            0
lat             8
long            8
numero_rua      0
bairro          0
cidade          0
piscina         0
gas_central     0
elevador        0
salao           0
academia        0
dtype: int64

In [44]:
# Pela alta presença de valores nulos na variável 'total' que represente a metragem total do apartamento, vamos excluir essa coluna tbm. 
df_limpo_itajai = df_limpo_itajai.drop('total', axis = 1)

In [45]:
# Verificamos no site do anúncio que o apartamento que possui valor nulo nos banheiros, na verdade tem 4 suítes. Colocamos a presença de 5 banheiros.
df_limpo_itajai.loc[132,'banheiros'] = 5

In [46]:
# Preenchemos os valores nulos das suítes com 0, pois quando não se tem suítes, o item não aparece no anúncio do site.
df_limpo_itajai['suites'].fillna(0, inplace = True)

In [47]:
# Definimos como 0 os apartamentos que possuem valores nulos nas vagas de carro, são apenas 3 imóveis e não deve impactar no modelo de qualquer forma.
df_limpo_itajai['vagas'].fillna(0, inplace = True)


In [48]:
# Converter os tipos de algumas colunas que estão em formato 'float' para 'int'. 
colunas_converter = ['suites','banheiros','vagas','privativos']
df_limpo_itajai[colunas_converter] = df_limpo_itajai[colunas_converter].astype('int64')
# Verifica os tipos de dados em cada coluna:
df_limpo_itajai.dtypes

titulo          object
preco          float64
quartos          int64
suites           int64
banheiros        int64
vagas            int64
privativos       int64
link            object
lat            float64
long           float64
numero_rua      object
bairro          object
cidade          object
piscina          int64
gas_central      int64
elevador         int64
salao            int64
academia         int64
dtype: object

In [49]:
# Determina o dataset final pós limpeza e tratamento dos dados:
df_final = df_limpo_itajai.copy()

In [50]:
# Salva o novo dataset:
df_final.to_csv('df_final.csv')